In [1]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

from getpass import getpass
import requests
import os

hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY
while "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:
    try: 
        assert not hard_reset
        response = requests.get("http://docker_router:8070/get_key").json()
        assert response.get('nvapi_key')
    except: response = {'nvapi_key' : getpass("NVIDIA API Key: ")}
    os.environ["NVIDIA_API_KEY"] = response.get("nvapi_key")
    try: requests.post("http://docker_router:8070/set_key/", json={'nvapi_key' : os.environ["NVIDIA_API_KEY"]}).json()
    except: pass
    hard_reset = False
    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")

print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")
from langchain_nvidia_ai_endpoints._common import NVEModel
NVEModel().available_models

NVIDIA API Key: ········
Retrieved NVIDIA_API_KEY beginning with "nvapi-ZAj..."


{'playground_nvolveqa_40k': '091a03bb-7364-4087-8090-bd71e9277520',
 'playground_nemotron_qa_8b': '0c60f14d-46cb-465e-b994-227e1c3d5047',
 'playground_llama2_70b': '0e349b44-440a-44e1-93e9-abe8dcb27158',
 'playground_nemotron_steerlm_8b': '1423ff2f-d1c7-4061-82a7-9e8c67afd43a',
 'playground_mistral_7b': '35ec3354-2681-4d0e-a8dd-80325dcf7c63',
 'playground_llama2_code_70b': '2ae529dc-f728-4a46-9b8d-2697213666d8',
 'playground_mamba_chat': '381be320-4721-4664-bd75-58f8783b43c7',
 'playground_starcoder2_15b': '6acada03-fe2f-4e4d-9e0a-e711b9fd1b59',
 'playground_gemma_2b': '5bde8f6f-7e83-4413-a0f2-7b97be33988e',
 'playground_sdxl': '89848fb8-549f-41bb-88cb-95d6597044a4',
 'playground_deplot': '3bc390c7-eeec-40f7-a64d-0c6a719985f7',
 'playground_neva_22b': '8bf70738-59b9-4e5f-bc87-7ab4203be7a0',
 'playground_nv_llama2_rlhf_70b': '7b3e3361-4266-41c8-b312-f5e33c81fc92',
 'playground_fuyu_8b': '9f757064-657f-4c85-abd7-37a7a9b6ee11',
 'playground_llama_guard': 'b34280ac-24e4-4081-bfaa-501e9ee16

----

<br>

## Part 1: Summary of RAG Workflows

This notebook will explore several paradigms and derive reference code to help you approach some of the most common retrieval-augmented workflows. Specifically, the following sections will be covered (with the differences highlighted):

<br>

> ***Vector Store Workflow for Conversational Exchanges:***
- Generate semantic embedding for each new conversation.
- Add the message body to a vector store for retrieval.
- Query the vector store for relevant messages to fill in the LLM context.

<br>

> ***Modified Workflow for an Arbitrary Document:***
- **Divide the document into chunks and process them into useful messages.**
- Generate semantic embedding for each **new document chunk**.
- Add the **chunk bodies** to a vector store for retrieval.
- Query the vector store for relevant **chunks** to fill in the LLM context.
    - ***Optional:* Modify/synthesize results for better LLM results.**

<br>

> **Extended Workflow for a Directory of Arbitrary Documents:**
- Divide **each document** into chunks and process them into useful messages.
- Generate semantic embedding for each new document chunk.
- Add the chunk bodies to **a scalable vector database for fast retrieval**.
    - ***Optional*: Exploit hierarchical or metadata structures for larger systems.**
- Query the **vector database** for relevant chunks to fill in the LLM context.
    - *Optional:* Modify/synthesize results for better LLM results.

<br>

Some of the most important terminology surrounding RAG is covered in detail on the [**LlamaIndex Concepts page**](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html), which itself is a great starting point for progressing towards the LlamaIndex loading and retrieving strategy. We highly recommend using it as a reference as you continue with this notebook and advise you to try out LlamaIndex after the course to consider the pros and cons firsthand!


## A conversation

In [2]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

In [3]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

embedder = NVIDIAEmbeddings(model="nvolveqa_40k")

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 332 ms, sys: 454 ms, total: 787 ms
Wall time: 2.97 s


In [4]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        page_content='[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard
many great things about them.'
    ),
    Document(
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

In [5]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        page_content='[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard
many great things about them.'
    ),
    Document(
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    )
]

### **Step 3:** Incorporating Conversation Retrieval Into Our Chain

Now that we have our loaded retriever component as a chain, we can incorporate it into our existing chat system as before. Specifically, we can start with an ***always-on RAG formulation*** where:
- **A retriever is always retrieving context by default**.
- **A generator is acting on the retrieved context**.

In [6]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

In [7]:
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

In [8]:
## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)
########################################################################

llm = ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser()

context_prompt = ChatPromptTemplate.from_messages([
    ('system',
        "Answer the question using only the context"
        "\n\nQuestion: {question}\n\nContext: {context}" ## Double reinforcement
    ), ('user', "{question}"),
])

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

Beras lives in the arctic.

In [9]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

The Rocky Mountains stretch across North America. (according to the context provided)

In [10]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

The Rocky Mountains are not close to California. They are a range of mountains that stretch across North America.

In [11]:
pprint(chain.invoke(
    "Where are the Rocky Mountains? Please include"
    " the author's reasoning, but provide more information!"
))

The Rocky Mountains are a range of mountains that stretch across North America. The information provided in the 
document suggests that the speaker, Beras, has not been to the Rocky Mountains before and is curious about them. 
The agent confirms the location of the Rocky Mountains and expresses a desire to explore more about their 
significance. Additionally, the agent suggests that Beras can learn more about the Rocky Mountains by doing 
research online or watching documentaries about them. The author's reasoning for stating that the Rocky Mountains 
are in North America is likely based on their geographical knowledge of the mountain range.

To provide more information, the Rocky Mountains extend from the northernmost part of British Columbia, in western 
Canada, to New Mexico in the Southwestern United States. The mountain range includes over 100 peaks that are more 
than 13,000 feet (3,962 meters) high. The Rocky Mountains are home to a diverse range of wildlife, including 
grizzly bears, elk, and mountain lions. The mountain range also provides important resources such as water, 
minerals, and timber. The Rocky Mountains are a popular destination for outdoor activities such as hiking, skiing, 
and camping.

In [12]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

The context does not provide information about the distance between Beras' location in the Arctic and the Rocky 
Mountains.

### **Step 4:** Automatic Conversation Storage

Now that we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.


In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

llm = ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser()

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "A user has asked a question: {input}\n\n Context: \n{context}\n\n"
    "Please continue the conversation by responding! Keep it brief and conversational!"
), ('user', '{input}')])

conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | llm})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, it's honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

That sounds like a great plan! I'm sure there are plenty of delicious ice cream shops in the Rocky Mountains. Have 
you decided which specific location you'd like to visit? We can also continue to chat about the Rocky Mountains and
their history if you'd like.

Based on your excitement about ice cream, I'm guessing it might be one of your favorite foods! Would you like to 
share some of your other favorite foods or any memorable ice cream experiences?

Oh, I must have misunderstood! You caught me! Honey is a delicious treat, too. Have you been to the Rocky 
Mountains, Beras? I've heard it's a beautiful place.

Based on our conversation, I think your favorite food might be ice cream! But to be sure, is there anything else 
you'd like to share about your favorite foods or any memorable experiences with them? 😊

----

<br>

## **Part 3 [Exercise]:** RAG For Document Chunk Retrieval

Given our prior exploration of document loading, the idea that data chunks can be embedded and searched through probably isn't surprising. With that said, it is definitely worth going over since applying RAG with documents is a double-edged sword; it may **seem** to work well out of the box but requires some extra care when optimizing it for truly reliable performance. It also provides an excellent opportunity to review some fundamental LCEL skills, so let's see what we can do!

<br>

### **Exercise:**

In the previous example, you may recall that we pulled in some relatively small papers with the help of [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv) using the following syntax:

```python
from langchain.document_loaders import ArxivLoader

docs = [
    ArxivLoader(query="2205.00445").load(),  ## MRKL
    ArxivLoader(query="2210.03629").load(),  ## ReAct
]
```

Given all that you've learned so far, choose a selection of papers that you would like to use and develop a chatbot that can talk about them!

<br>

Though this is a pretty big task, a walkthrough of ***most*** of the process will be provided below. By the end of the walkthrough, many of the necessary puzzle pieces will be provided, and your real task will be to integrate them together for the final `retrieval_chain`. When you're done, get ready to re-integrate the chain (or a flavor of your choice) in the last notebook as part of the evaluation exercise!


In [14]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

In [15]:
## TODO: Please pick some papers and add them to the list as you'd like
## NOTE: To re-use for the final assessment, make sure at least one paper is < 1 month old
print("Loading Documents")
docs = [
    ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ## Some longer papers
    # ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    # ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    # ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
    ## TODO: Feel free to add more
]

Loading Documents


In [16]:
## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = doc[0].page_content
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

Chunking Documents


In [17]:
## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - Metadata: {chunks[0].metadata}")
    print(f" - # Chunks: {len(chunks)}")
    print()

Available Documents:
 - Attention Is All You Need
 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
 - MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge 
sources and discrete reasoning
 - Mistral 7B
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena 

Document 0
 - Metadata: {'Published': '2023-08-02', 'Title': 'Attention Is All You Need', 'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin', 'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 English-to-Frenc

## Construct Your Document Vector Stores

In [18]:
%%time
## ^^ This cell will output a time
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

from langchain_core.prompts import ChatPromptTemplate

embedder = NVIDIAEmbeddings(model="nvolveqa_40k", model_type=None)

## Construct series of document vector stores
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: user 549 ms, sys: 22.1 ms, total: 571 ms
Wall time: 32 s


From there, we can combine our indices into a single one using the following utility:

In [19]:
embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

if 'docstore' not in globals():
    ## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
    docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 225 chunks


## Implement Your RAG Chain

In [31]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from gradio import gr

from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.vstores import default_FAISS

llm = ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser()
convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

retrieval_chain = (
    {'input': RunnableAssign(itemgetter('input'))},
    {'history': RunnableAssign(itemgetter('history')), 'context': RunnableAssign(itemgetter('context'))}
)

stream_chain = chat_prompt | llm

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    # First, perform the retrieval based on the input message
    retrieval = {key: function({'input': message, 'history': history, 'context': context}) for key, function in retrieval_chain.items()}
    line_buffer = ""

    # Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        # If you're using standard print, keep the line from getting too long
        if not return_buffer:
            line_buffer += token
            if "\n" in line_buffer:
                line_buffer = ""
            if ((len(line_buffer) > 84 and token and token[0] == " ") or len(line_buffer) > 100):
                line_buffer = ""
                yield "\n"
                token = "  " + token.lstrip()
        yield buffer if return_buffer else token

    # Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)

# Start of Agent Event Loop
test_question = "Tell me about RAG!"  # <- modify as desired

# Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')


ImportError: cannot import name 'gr' from 'gradio' (/home/ranga/.local/lib/python3.10/site-packages/gradio/__init__.py)

In [ ]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

## SAVING

In [29]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


In [30]:
## Assuming you haven't imported these yet
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

embedder = NVIDIAEmbeddings(model="nvolveqa_40k")
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder)
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss
80%, on par with the level of agreement among human experts, establishing a foundation for an
LLM-based evaluation framework.
Acknowledgement
This project is partly supported by gifts from Anyscale, Astronomer, Google, IBM, Intel, Lacework,
Microsoft, MBZUAI, Samsung SDS, Uber, and VMware. Lianmin Zheng is supported by a Meta
Ph.D. Fellowship. We extend our thanks to Xinyang Geng, Hao Liu, Eric Wallace, Xuecheng Li,
Tianyi Zhang, Qirong Ho, and Kevin Lin for their insightful discussions.
